# Perform Queries in our Graph

In [23]:
from rdflib import Graph
g = Graph()
g.parse("../IFC Conversion/house.ttl")

g

<Graph identifier=N1d05fbcbd95d4ddcae98ca5bae113079 (<class 'rdflib.graph.Graph'>)>

## Using SPARQL

### Art. 22§2

The doors should not have a width below 1050mm.

In [24]:
ourQuery = """
PREFIX bot: <https://w3id.org/bot#>
PREFIX beo: <https://w3id.org/beo#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?s ?w
WHERE{ ?s rdf:type beo:Door .
    ?s ex:width ?w .
    FILTER (?w < 1050)
}
"""

qres = g.query(ourQuery)

print( f"Found {len(qres)} Doors with Width below 1050 mm: " )
for row in qres:
    print(f"{row.s} - door width = {row.w}")

Found 17 Doors with Width below 1050 mm: 
https://example.org#M_Single-Flush_0915x2032mm_398822 - door width = 915.0
https://example.org#M_Single-Flush_0915x2032mm_398909 - door width = 915.0
https://example.org#M_Single-Flush_0915x2032mm_399431 - door width = 915.0
https://example.org#M_Single-Flush_0915x2032mm_399501 - door width = 915.0
https://example.org#M_Single-Flush_0915x2032mm_399643 - door width = 915.0
https://example.org#M_Single-Flush_0915x2032mm_399925 - door width = 915.0
https://example.org#M_Single-Flush_0915x2032mm_400081 - door width = 915.0
https://example.org#M_Single-Flush_0915x2134mm_358185 - door width = 915.0
https://example.org#M_Single-Flush_0915x2134mm_358480 - door width = 915.0
https://example.org#M_Single-Flush_0915x2134mm_359949 - door width = 915.0
https://example.org#M_Single-Flush_0915x2134mm_377662 - door width = 915.0
https://example.org#M_Single-Flush_0915x2134mm_378185 - door width = 915.0
https://example.org#M_Single-Flush_0915x2134mm_378243 - do

### Art. 20§2

Stairs can have at most 17 risers.

In [25]:
ourQuery = """
# Check Riser Height
PREFIX ex: <https://example.org#>
PREFIX beo: <https://w3id.org/beo#>
select * where {
    ?s ?p beo:Stair .
    ?s ex:NumberOfRailings ?rl .
    ?s ex:NumberOfRiser ?ri .
    ?s ex:RiserHeight ?rh .
    ?s ex:TreadLength ?tl
    FILTER(?ri > 17)
} limit 100
"""

qres = g.query(ourQuery)

if len(qres) > 0:
    print( "Found these Stair(s) with more than 17 risers:" )
    for row in qres:
        print(f"{row.s} : Risers = {row.ri}")
else:
    print( "No stairs have more than 17 risers." )

Found these Stair(s) with more than 17 risers:
https://example.org#PrecastStair_Stair_383689 : Risers = 18


### Art. 20§3

The height of risers may be at most 18cm.

In [26]:
ourQuery = """
# Check Riser Height
PREFIX ex: <https://example.org#>
PREFIX beo: <https://w3id.org/beo#>
select * where {
    ?s ?p beo:Stair .
    ?s ex:NumberOfRailings ?r .
    ?s ex:RiserHeight ?rh .
    ?s ex:TreadLength ?tl
    FILTER(?rh > 180)
} limit 100
"""

qres = g.query(ourQuery)

if len(qres) > 0:
    print( "Found these Stairs too high risers:" )
    for row in qres:
        print(f"{row.s} : RH = {row.rh} TL = {row.tl}")
else:
    print( "All stairs have a small enough riser height." )

All stairs have a small enough riser height.


The length of treads should be at least 23cm.

In [27]:
ourQuery = """
# Check Riser Height
PREFIX ex: <https://example.org#>
PREFIX beo: <https://w3id.org/beo#>
select * where {
    ?s ?p beo:Stair .
    ?s ex:NumberOfRailings ?r .
    ?s ex:RiserHeight ?rh .
    ?s ex:TreadLength ?tl
    FILTER(?tl < 230)
} limit 100
"""

qres = g.query(ourQuery)

if len(qres) > 0:
    print( "Found these Stairs with too short tread lengths:" )
    for row in qres:
        print(f"{row.s} : RH = {row.rh} TL = {row.tl}")
else:
    print( "All stairs have a sufficient tread length." )

All stairs have a sufficient tread length.


### Art. 20§3
The stairs and the calculated stair formula: 2xRiser + 1x TreadLength

In [28]:
ourQuery = """
# Check Stair formula
PREFIX ex: <https://example.org#>
PREFIX beo: <https://w3id.org/beo#>
select * where {
    ?s ?p beo:Stair .
    ?s ex:NumberOfRailings ?r .
    ?s ex:RiserHeight ?rh .
    ?s ex:TreadLength ?tl
    BIND( (2 * ?rh + ?tl) AS ?StairFormula )
    FILTER((?StairFormula < 590) || (?StairFormula > 630))
} limit 100
"""

qres = g.query(ourQuery)

if len(qres) > 0:
    print( "Found these Stairs with non-compliant Stair Formula: " )
    for row in qres:
        print(f"{row.s} : 2xRH+TL = {row.StairFormula}")
else:
    print("All stairs comply with the Stair Formula")

Found these Stairs with non-compliant Stair Formula: 
https://example.org#PrecastStair_Stair_397996 : 2xRH+TL = 454


## Now using SHACL

In [29]:
!pip install pyshacl

### Load a TTL graph and validate it

In [30]:
from pyshacl import validate
from os import path

# Load our Model Data (TTL)
data_ttl_file = path.abspath("../IFC Conversion/house.ttl")

# Validate (but not shapes have been defined)
conforms, v_graph, v_text = validate(
    data_ttl_file, shacl_graph=None, inference='rdfs',
    serialize_report_graph=True)

# Output
print(conforms)
print(v_graph)
print(v_text)

True
b'@prefix ns1: <http://www.w3.org/ns/shacl#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n[] a ns1:ValidationReport ;\n    ns1:conforms true .\n\n'
Validation Report
Conforms: True



## Now with SHACL shapes

You can always test in the SHACL playground...


In [33]:
shape_query = '''
@prefix dash: <http://datashapes.org/dash#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

@prefix beo: <https://w3id.org/beo#> .
@prefix ex: <https://example.org#> .

ex:DoorShape
    a sh:NodeShape ;
    sh:targetClass beo:Door ;
    sh:property [
        sh:path ex:height ;
        sh:datatype xsd:double ;
        sh:minInclusive 2090 ;
        # sh:maxInclusive 1000 ;
        sh:message "Art.22§1 Doors should be at least 2.09m high." ;
    ] .

ex:DoorShape
    a sh:NodeShape ;
    sh:targetClass beo:Door ;
    sh:property [
        sh:path ex:width ;
        sh:datatype xsd:double ;
        sh:minInclusive 1050 ;
        # sh:maxInclusive 1000 ;
        sh:message "Art.22§2 Doors should be at least 1.05m wide." ;
    ] .


ex:StairShape
    a sh:NodeShape ;
    sh:targetClass beo:Stair ;
    sh:property [
        sh:path ex:NumberOfRiser ;
        sh:maxExclusive 18 ;
        sh:datatype xsd:integer ;
        sh:message "Need at most 17 risers";
    ] .
'''

conforms, v_graph, v_text = validate(
    data_ttl_file,
    shacl_graph=shape_query,
    inference='rdfs',
    serialize_report_graph=True
)
print(conforms)
print(v_graph)
print(v_text)

False
b'@prefix ex: <https://example.org#> .\n@prefix sh: <http://www.w3.org/ns/shacl#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n[] a sh:ValidationReport ;\n    sh:conforms false ;\n    sh:result [ a sh:ValidationResult ;\n            sh:focusNode ex:PrecastStair_Stair_383689 ;\n            sh:resultMessage "Need at most 17 risers" ;\n            sh:resultPath ex:NumberOfRiser ;\n            sh:resultSeverity sh:Violation ;\n            sh:sourceConstraintComponent sh:MaxExclusiveConstraintComponent ;\n            sh:sourceShape [ sh:datatype xsd:integer ;\n                    sh:maxExclusive 18 ;\n                    sh:message "Need at most 17 risers" ;\n                    sh:path ex:NumberOfRiser ] ;\n            sh:value 18 ],\n        [ a sh:ValidationResult ;\n            sh:focusNode ex:M_Single-Flush_0915x2032mm_399643 ;\n            sh:resultMessage "Art.22\xc2\xa72 Doors should be at least 1.05m wide." ;\n            sh:resultPath ex:width ;\n            sh:r